In [1]:
import pandas as pd
import os 
import numpy as np




In [13]:
import os
import pandas as pd
import re

base_path = "./"

def parse_folder_name(name):
    parts = name.split("_")
    gas = parts[1]
    temp = parts[2].replace("K","")  # 온도 숫자만
    # 압력 범위 추출
    pressure_tokens = [p for p in parts if re.match(r"^\d+(\.\d+)?$", p)]
    low_p, high_p = None, None
    if len(pressure_tokens) >= 2:
        low_p, high_p = pressure_tokens[0], pressure_tokens[-1]
    struct_info, sampling, model = None, None, None
    if "_dataset_" in name:
        after = name.split("_dataset_")[-1]
        tokens = after.split("__")
        struct_info = tokens[0]
        if len(tokens) > 1:
            sampling = tokens[1]
        if len(tokens) > 2:
            model = tokens[2]
    return gas, float(temp), float(low_p), float(high_p), struct_info, sampling, model

rows = []
grid_folders = [f for f in os.listdir(base_path) if f.startswith("GRID_RUNS_") and os.path.isdir(os.path.join(base_path,f))]

for folder in grid_folders:
    gas, temp, low_p, high_p, struct_info, sampling, model = parse_folder_name(folder)
    folder_path = os.path.join(base_path, folder)
    row = {
        "gas": gas,
        "temperature": temp,
        "low_pressure": low_p,
        "high_pressure": high_p,
        "struct_info": struct_info,
        "sampling": sampling,
        "model": model
    }
    for trial in os.listdir(folder_path):
        trial_path = os.path.join(folder_path, trial)
        if os.path.isdir(trial_path) and trial.startswith("trial_"):
            for file in os.listdir(trial_path):
                if file.startswith("metrics_holdout") and file.endswith(".csv"):
                    df = pd.read_csv(os.path.join(trial_path, file))
                    if not df.empty:
                        metrics = df.iloc[0].to_dict()
                        for k,v in metrics.items():
                            if k in ["R2","MAE","RMSE","MAPE_percent"]:
                                row[f"{trial}_{k}"] = v
    rows.append(row)

final_df = pd.DataFrame(rows)
final_df.to_csv("aggregated_metrics_clean.csv", index=False)
print("Saved aggregated_metrics_clean.csv")


ValueError: could not convert string to float: 'Ar'

In [ ]:
parse_folder_name(folder)

In [12]:
final_df

,gas,temperature,pressure_range,struct_info,sampling,model,trial_001_Model,trial_001_R2,trial_001_MAE,trial_001_RMSE,...,trial_004_predict_time_sec,trial_005_Model,trial_005_R2,trial_005_MAE,trial_005_RMSE,trial_005_MAPE_percent,trial_005_n_train,trial_005_n_test,trial_005_fit_time_sec,trial_005_predict_time_sec
0,RUNS,Ar,273K_Ar_273,struct+input,qt_then_rd,cat,cat,0.961278,0.102850,0.025910,...,0.002292,cat,0.953047,0.106913,0.030560,5.532962,6213,1553,3.491001,0.002300
1,RUNS,Ar,273K_Ar_273,struct+input,qt_then_rd,gbm,gbm,0.959874,0.103147,0.026850,...,0.012245,gbm,0.952513,0.106706,0.030908,5.259507,6213,1553,15.541272,0.012069
2,RUNS,Ar,273K_Ar_273,struct+input,qt_then_rd,rf,rf,0.955179,0.101437,0.029992,...,0.280853,rf,0.947754,0.104827,0.034006,5.129294,6213,1553,35.991968,0.282099
3,RUNS,Ar,273K_Ar_273,struct+input,random_struct,cat,cat,0.935458,0.123137,0.062272,...,0.002326,cat,0.942212,0.118484,0.057130,6.174892,6213,1553,3.506600,0.002307
4,RUNS,Ar,273K_Ar_273,struct+input,random_struct,gbm,gbm,0.941161,0.120080,0.056769,...,0.012683,gbm,0.940388,0.119701,0.058933,6.157140,6213,1553,15.774862,0.012956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,RUNS,O2,293K_O2_293,struct+input,random_with_input,gbm,gbm,0.626585,0.080120,0.382458,...,0.012093,gbm,0.614787,0.085330,0.394303,4.808026,5690,1423,14.180165,0.011982
104,RUNS,O2,293K_O2_293,struct+input,random_with_input,rf,rf,0.628784,0.080471,0.380206,...,0.266258,rf,0.625758,0.084096,0.383073,4.605734,5690,1423,32.723660,0.265604
105,RUNS,O2,293K_O2_293,struct,random_struct,cat,cat,0.200664,0.342934,0.818693,...,0.002163,cat,0.196103,0.338099,0.822867,31.684699,5690,1423,3.080801,0.002145
106,RUNS,O2,293K_O2_293,struct,random_struct,gbm,gbm,0.198309,0.345314,0.821105,...,0.011071,gbm,0.187832,0.347843,0.831333,32.659289,5690,1423,12.478467,0.011061


In [ ]:
import os
import re
import pandas as pd

base_path = "./"

# Helper function to parse folder name into attributes
def parse_folder_name(name):
    # Example: GRID_RUNS_Ar_273K_Ar_273_0.01_to_Ar_273_15_dataset_struct+input__qt_then_rd__cat
    parts = name.split("_")
    gas = parts[1]   # Ar
    temp = parts[2]  # 273K
    pressure_range = parts[3] + "_" + parts[4] + "_" + parts[5]  # Ar_273_0.01_to_Ar_273_15_dataset
    struct_info = None
    sampling = None
    model = None
    
    if "__" in name:
        after_dataset = name.split("_dataset_")[-1]
        tokens = after_dataset.split("__")
        struct_info = tokens[0]  # struct or struct+input
        if len(tokens) > 1:
            sampling = tokens[1]  # qt_then_rd, random_struct, etc.
        if len(tokens) > 2:
            model = tokens[2]  # cat, gbm, rf
    return gas, temp, pressure_range, struct_info, sampling, model

metrics_dict = {}

# Scan GRID_RUNS_* folders
grid_folders = [f for f in os.listdir(base_path) if f.startswith("GRID_RUNS") and os.path.isdir(os.path.join(base_path,f))]

for folder in grid_folders:
    gas, temp, pressure_range, struct_info, sampling, model = parse_folder_name(folder)
    folder_path = os.path.join(base_path, folder)
    
    for trial in os.listdir(folder_path):
        trial_path = os.path.join(folder_path, trial)
        if os.path.isdir(trial_path) and trial.startswith("trial_"):
            for file in os.listdir(trial_path):
                if file.startswith("metrics_holdout") and file.endswith(".csv"):
                    metrics_file = os.path.join(trial_path, file)
                    df = pd.read_csv(metrics_file)
                    # Expect single row
                    if not df.empty:
                        key = (gas, temp, pressure_range, struct_info, sampling, model)
                        if key not in metrics_dict:
                            metrics_dict[key] = {}
                        metrics_dict[key][trial] = df.iloc[0].to_dict()

# Now build a DataFrame with trials as columns of metrics
rows = []
for key, trial_data in metrics_dict.items():
    gas, temp, pressure_range, struct_info, sampling, model = key
    row = {
        "gas": gas,
        "temperature": temp,
        "pressure_range": pressure_range,
        "struct_info": struct_info,
        "sampling": sampling,
        "model": model
    }
    # For each trial, flatten its metrics into subcolumns
    for trial, metrics in trial_data.items():
        for metric_name, metric_value in metrics.items():
            col_name = f"{trial}_{metric_name}"
            row[col_name] = metric_value
    rows.append(row)

final_df = pd.DataFrame(rows)

# import caas_jupyter_tools
# caas_jupyter_tools.display_dataframe_to_user("Aggregated Trial Metrics", final_df)


IndexError: list index out of range

In [7]:
parse_folder_name(folder)

IndexError: list index out of range